In [1]:
import ccxt
import pandas as pd
import numpy as np

# Initialize Binance
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1h'  # hourly candles
limit = 1000  # number of candles to fetch

# Fetch OHLCV data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)


In [2]:
# ===============================
# 2. Calculate RSI
# ===============================
import ta
df["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()


In [3]:
# ===============================
# 3. MACD Calculation
# ===============================
df["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
df["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
df["MACD"] = df["EMA12"] - df["EMA26"]
df["Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
df["Histogram"] = df["MACD"] - df["Signal"]


In [4]:
import talib
df["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)

In [5]:
# ------------------------------
# 2. Calculate SMA (3 closes)
# ------------------------------
sma_window = 3
df['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)


In [6]:
# ------------------------------
# 2. Compute TSI
# ------------------------------
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi

df['TSI'] = compute_tsi(df['close'])


In [7]:
# ========================
# 2. Stochastic Oscillator
# ========================
period = 14
smooth_k = 3
smooth_d = 3

lowest_low = df["low"].rolling(period).min()
highest_high = df["high"].rolling(period).max()

df["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
df["%K"] = df["%K"].rolling(smooth_k).mean()
df["%D"] = df["%K"].rolling(smooth_d).mean()


In [8]:
# ========================
# 3. Swing Highs / Lows
# ========================
lookback_swings = 5

def find_swings(prices, lb):
    labels = [0]*len(prices)
    for i in range(lb, len(prices)-lb):
        if prices[i] == max(prices[i-lb:i+lb+1]):
            labels[i] = -1   # swing high
        elif prices[i] == min(prices[i-lb:i+lb+1]):
            labels[i] = 1    # swing low
    return labels

df["Label"] = find_swings(df["close"].values, lookback_swings)


In [9]:
df.dropna(inplace=True)

In [10]:
df['Label'] = df['Label'] + 1

In [11]:
print(df.columns.tolist())

['open', 'high', 'low', 'close', 'volume', 'RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D', 'Label']


In [12]:
df.to_csv('binance_btc_usdt_1h_indicators.csv')

In [26]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
from sklearn.preprocessing import MinMaxScaler

FEATURES = ['RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D']

X = df[FEATURES].values
y = df["Label"].values

# ===============================
# 2. Scale features
# ===============================
# Normalize
scaler = MinMaxScaler()
X = scaler.fit_transform(X)


In [27]:
# Reshape into sequences for Transformer
SEQ_LEN = 60
def create_sequences(X, y, seq_len=SEQ_LEN):
    xs, ys = [], []
    for i in range(len(X)-seq_len):
        xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(xs), np.array(ys)

X_seq, y_seq = create_sequences(X, y, SEQ_LEN)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, shuffle=False
)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [30]:
import torch.nn as nn

num_classes = len(np.unique(y_seq))

class TransformerModel(nn.Module):
    def __init__(self, feature_size, num_classes=num_classes, num_layers=4, hidden_dim=128, nhead=8, dropout=0.2):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(feature_size, hidden_dim)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=nhead, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.fc = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, num_classes)
        )
        
    def forward(self, x):
        x = self.embedding(x)   # [batch, seq, hidden_dim]
        x = self.transformer(x) # [batch, seq, hidden_dim]
        x = x.mean(dim=1)       # global average pooling
        return self.fc(x)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TransformerModel(feature_size=X_train.shape[2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/20, Loss: 0.5790
